## Intorduction

The purpose of this projects is....


### Imports


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_games = pd.read_csv('datasets/games.csv')
df_games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [3]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


#### Import Conclusions:

There are several null values in the ratings, user_score, and critic_scores.  These will need to be handled, but we need to preserve some way of telling what games did not have a score.  The best option is to provide a non-conflicting value like -1 since no user/critic score can be -1 it makes it clear that this game did not have a rating.  However, this will make aggregation analysis less accurate so we will keep the original data as is so we can make aggregations off of that.  

The column names are all capitalized so we need to make them lowercase.  The column types seem appropriate for the data they hold.  The only noticeable issue is the year of release which is a float, but since we are not looking at partial years it should be either an int or a datetime type.  However, there are some null values in the year column so we need to handle them first and an int would be the best option where the null values are replaced by a -1 so it is noticeable in the data.  This should not affect the data too much, but in the case we need to do data aggregation it will cause issues like above so we will ensure this is in a separate data set as well. 

In [6]:
df_games.columns = df_games.columns.str.lower()
df_games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [10]:
df_games_no_null = df_games.copy()
df_games_no_null['year_of_release'] = df_games['year_of_release'].fillna(-1).astype('int')
df_games_no_null['critic_score'] = df_games['critic_score'].fillna(-1)
df_games_no_null['user_score'] = df_games['user_score'].fillna(-1)
df_games_no_null['rating'] = df_games['rating'].fillna('NA')

## Only 2 that are missing we will mark them with NA for now
df_games_no_null['genre'] = df_games['genre'].fillna('NA')
df_games_no_null['name'] = df_games['genre'].fillna('NA')

df_games_no_null.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16715 non-null  int32  
 3   genre            16715 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     16715 non-null  float64
 9   user_score       16715 non-null  object 
 10  rating           16715 non-null  object 
dtypes: float64(5), int32(1), object(5)
memory usage: 1.3+ MB
